In [ ]:
pip install matplotlib

In [ ]:
pip install seaborn

In [57]:
# Bibliotecas

import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np
import csv
import time
import string
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
''' Funciones '''

In [58]:
# Esta función calcula la frecuencia del registro y su porcentaje sobre la totalidad del df

def resumen_cant_porcentaje(df, columna):
    # Cuanta la cantidad de True/False luego calcula el porcentaje
    counts = df[columna].value_counts()
    percentages = round(100 * counts / len(df),2)
    # Crea un dataframe con el resumen
    df_results = pd.DataFrame({
        "Cantidad": counts,
        "Porcentaje": percentages
    })
    return df_results

In [59]:
# Esta función calcula el rating del video dependiendo del sentimiento y la recomendación

def calcula_rating(row):
    if row["sentiment_analysis"] == 0 and not row["reviews_recommend"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["reviews_recommend"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["reviews_recommend"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["reviews_recommend"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["reviews_recommend"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["reviews_recommend"]:
        return 5
    else:
        return None

El objetivo del modelo es generar una lista de 5 juegos al proporcionar el nombre de un juego o el ID de un usuario. La recomendación se basará en puntajes de cada juego que se calcularán dependiento del sentimiento y si se recomendaria o no

Se genera la siguiente tabla de rating para hacer una nueva columna que indique estos puntajes. son solo 5 por que si el sentimiento es negativo pues de una no se recomendaría

1: Sentimiento negativo, ya sea recomendado o no.
2: Sentimiento neutral y no recomendado.
3: Sentimiento neutral y recomendado.
4: Sentimiento positivo y no recomendado.
5: Sentimiento positivo y recomendado.

In [ ]:
# Inicialment debo abrir df_items y df_reviews que ya se analizaron en sus notebooks

In [ ]:
# Ruta al archivo df_reviews.parquet
ruta_reviews = r"C:\Users\Usuario\Henry\PI1_ML\Modelo_Aprendizaje_Automatico\Datos_Modelo\df_reviews.parquet"

# Leer el archivo CSV y convertirlo en un DataFrame
df_reviews = pd.read_parquet(ruta_reviews)

df_reviews.head()

In [61]:
# Ruta al archivo df_items.parquet
ruta_items = r"C:\Users\Usuario\Henry\PI1_ML\Modelo_Aprendizaje_Automatico\Datos_Modelo\df_items.parquet"

# Leer el archivo CSV y convertirlo en un DataFrame
df_items = pd.read_parquet(ruta_items)

df_items.head()

,user_id,items_count,user_url,item_id,item_name,playtime_forever
0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0
1,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0
2,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0
3,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0
4,76561197970982479,277,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0


In [ ]:
# Aca se  debe crear una columna de raiting pero para eso se creará primero una función que me calcule la frecuencia 
# del registro y su porcentaje sobre la totalidad de registros (La función se encuentra al principio del archivo)

In [ ]:
# Se crea una columna nueva en df_reviews con columna raiting

df_reviews['rating'] = df_reviews.apply(calcula_rating, axis=1)
df_reviews.head()

In [63]:
# Borro todas las columnas menos 'user_id', 'reviews_item_id' y 'rating'

columnas_borrar = ['user_url',	'reviews_date',	'reviews_recommend',	'sentiment_analysis',	'reviews_helpful']
df1 = df_reviews.drop(columnas_borrar, axis=1)


In [64]:
df1.head()

,user_id,reviews_item_id,rating
9,76561197970982479,1250,5
19,76561197970982479,1250,5
21,76561197970982479,1250,5
22,76561197970982479,1250,5
29,76561197970982479,1250,5


In [65]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1685956 entries, 9 to 43706881
Data columns (total 3 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   user_id          1685956 non-null  object
 1   reviews_item_id  1685956 non-null  object
 2   rating           1685956 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 51.5+ MB


In [66]:
# Cambio el nombre reviews_item_id  por item_id para hacer merge

df1 = df1.rename(columns={'reviews_item_id': 'item_id'})

In [ ]:
df1.head()

In [ ]:
# Ahora hago un df2 basado en df_items que contenga las columnas Se crea otro df que contenga unicamente los 'item_id' y los nombres de los juego,

df_items.head()

In [69]:
columnas_borrar_items = ['user_id',	'items_count',	'user_url',	'playtime_forever']
df2 = df_items.drop(columnas_borrar_items , axis=1)

In [ ]:
df2.head()

In [71]:
df2 = df2.dropna

In [15]:
df2.shape

(5063830, 2)

In [25]:
df2.shape

(10979, 2)

In [ ]:
# Cuento el numero de juegos que debe ser el largo del df por que ya no tienerepetidos ni na

#cantidad_juegos = len(df2)
#cantidad_juegos


In [ ]:
# hago merge entre los dos con item_id

df1 = pd.merge(df1, df2, on='item_id', how='inner')

In [22]:
df1.head()

AttributeError: 'function' object has no attribute 'head'

In [172]:
# Necesito saber el tipo de objetos en cada columna por que no me grafica

df1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374770 entries, 0 to 1374769
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1374770 non-null  object
 1   item_id    1374770 non-null  object
 2   rating     1374770 non-null  int64 
 3   item_name  1374770 non-null  object
dtypes: int64(1), object(3)
memory usage: 42.0+ MB


In [174]:
# Voy aver si cambiando objetc por stn

df1['item_name'] = df1['item_name'].astype(str)


In [175]:
df1.info()
# no cambia el tipo de dato

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374770 entries, 0 to 1374769
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1374770 non-null  object
 1   item_id    1374770 non-null  object
 2   rating     1374770 non-null  int64 
 3   item_name  1374770 non-null  object
dtypes: int64(1), object(3)
memory usage: 42.0+ MB


In [169]:
# Es importante revisar las variables de impacto para definir como hacer el modelo
# Asi que hahago el siguiente gráfico para visualizarlo bien!
# rimero agrupo las calificaciones por rating

ratings_count = df1['rating'].value_counts().reset_index()
ratings_count.columns = ['rating', 'count']

In [167]:
ratings_count

,rating,count
0,5,706148
1,1,300094
2,3,279048
3,4,50794
4,2,38686


In [170]:
# Agrupa las calificaciones por rating
ratings_count = df1['rating'].value_counts().reset_index()
ratings_count.columns = ['rating', 'count']
# Agrupa los juegos únicos por rating
unique_games_per_rating = df1.groupby('rating')['item_name'].nunique().reset_index()

In [171]:
unique_games_per_rating

# problema....item_name no me sale..creo que cambiaré el tipo de objeto en la columna 

,rating,item_name
0,1,1707
1,2,512
2,3,1214
3,4,698
4,5,2193


In [ ]:
# Crea una figura con dos subplots

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

In [164]:
# Primer subplot
sns.barplot(data=ratings_count, x='rating', y='count', ax=axes[0])
axes[0].set_title('Cantidad de reviews según los ratings')
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Cantidad revies')
# Segundo subplot
sns.barplot(data=unique_games_per_rating, x='rating', y='item_name', ax=axes[1])
axes[1].set_title('Cantidad de Juegos por rating')
axes[1].set_xlabel('Rating')
axes[1].set_ylabel('Cantidad de juegos')
# Ajusta y muestra los graficos
plt.tight_layout()
plt.show()

<Figure size 640x480 with 0 Axes>

In [ ]:
# Por ningún lado me grafica esto.....no se si sea por el tipo de dato

In [179]:
# Igual aunque no pude graficar para analizar los 
# datos voy a cambiar del nonmbre df por df_modelo para incluirlo en el modelo como tal

df_recomendacion = df1

In [180]:
df_recomendacion.head()

,user_id,item_id,rating,item_name
0,76561197970982479,1250,5,Killing Floor
1,76561197970982479,1250,5,Killing Floor
2,76561197970982479,1250,5,Killing Floor
3,76561197970982479,1250,5,Killing Floor
4,76561197970982479,1250,5,Killing Floor
